In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname('__file__'), '..')))
from src.encoding.tools import create_encoding_pipeline
from src.encoding.encoders import *
import pandas as pd

from sklearn import set_config

In [2]:
import numpy as np

data = pd.DataFrame({
    'num_feature1': [1.0, 2.0, 3.0, 4.0],
    'num_feature2': [10.0, 20.0, 30.0, 40.0],
    'cat_feature1': ['A', 'B', 'A', 'B'],
    'cat_feature2': ['X', 'Y', 'X', 'Y'],
    'date_feature': pd.to_datetime(['2021-01-01', '2021-01-27', '2021-01-03', '2021-03-26'])
})

y = pd.DataFrame({
    'target1': np.random.rand(4),
    'target2': np.random.rand(4)
})

data['num_feature1'] = data['num_feature1'].astype('float')
data['num_feature2'] = data['num_feature2'].astype('float')
data['cat_feature1'] = data['cat_feature1'].astype('category')
data['cat_feature2'] = data['cat_feature2'].astype('category')
data['date_feature'] = pd.to_datetime(data['date_feature'])

In [3]:
test_data = pd.DataFrame({
    'num_feature1': [5.0],
    'num_feature2': [50.0],
    'cat_feature1': ['A'],
    'cat_feature2': ['X'],
    'date_feature': pd.to_datetime(['2021-03-27'])
})

test_data['num_feature1'] = test_data['num_feature1'].astype('float')
test_data['num_feature2'] = test_data['num_feature2'].astype('float')
test_data['cat_feature1'] = test_data['cat_feature1'].astype('category')
test_data['cat_feature2'] = test_data['cat_feature2'].astype('category')
test_data['date_feature'] = pd.to_datetime(test_data['date_feature'])

In [4]:
encoders_dict = {
    'number': {
        'as_number': {
            'imputers': [imputers.SimpleImputer(strategy='mean')],
            'encoders': [
                ne.StandardScaler(),
            ]
        }
    },
    'category': {
        'as_category': {
            'imputers': [imputers.SimpleImputer(strategy='most_frequent')],
            'encoders': [
                ne.TargetEncoder(target_type='continuous-multioutput'),
            ]
        }
    },
    'datetime': {
        'as_number': {
            'imputers': [de.DateFeatureExtractor()],
            'encoders': [
                ne.CyclicalFeatures(drop_original=True)
            ]
        },
        'as_category': {
            'imputers': [de.DateFeatureExtractor(dtype='category')],
            'encoders': [
                ne.TargetEncoder(),
            ]
        }
    }
}

In [5]:
processor = create_encoding_pipeline(encoders_dict)

Creating encoding pipeline


In [6]:
processor

FeatureUnion(transformer_list=[('columntransformer-1',
                                ColumnTransformer(transformers=[('pipeline',
                                                                 Pipeline(steps=[('featureunion-1',
                                                                                  FeatureUnion(transformer_list=[('simpleimputer',
                                                                                                                  SimpleImputer())])),
                                                                                 ('featureunion-2',
                                                                                  FeatureUnion(transformer_list=[('standardscaler',
                                                                                                                  StandardScaler())]))]),
                                                                 <sklearn.compose._column_transformer.make_column_selector obj...
                                ColumnTransformer(transformers=[('pipeline',
                                                                 Pipeline(steps=[('featureunion-1',
                                                                                  FeatureUnion(transformer_list=[('datefeatureextractor',
                                                                                                                  DateFeatureExtractor(dtype='category'))])),
                                                                                 ('featureunion-2',
                                                                                  FeatureUnion(transformer_list=[('targetencoder',
                                                                                                                  TargetEncoder())]))]),
                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x77edfb287850>)]))],
             verbose=True)

In [7]:
for target in y.columns:
    processor.fit(data, y)
    processor

[FeatureUnion]  (step 1 of 4) Processing columntransformer-1, total=   0.0s


InvalidParameterError: The 'target_type' parameter of TargetEncoder must be a str among {'multiclass', 'continuous', 'auto', 'binary'}. Got 'continuous-multioutput' instead.

In [ ]:
encoded = processor.transform(data)
print(encoded)

In [ ]:
encoded = processor.transform(test_data)
print(encoded)